In [8]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import tensorflow as tf
import tensorflow
import gym


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape
from tensorflow.keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

In [10]:
env = gym.make("Taxi-v3").env

In [11]:
action_size = env.action_space.n

model = Sequential()
model.add(Embedding(500, 10, input_length=1))
model.add(Reshape((10,)))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(action_size, activation='linear'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1, 10)             5000      
                                                                 
 reshape_1 (Reshape)         (None, 10)                0         
                                                                 
 dense_4 (Dense)             (None, 50)                550       
                                                                 
 dense_5 (Dense)             (None, 50)                2550      
                                                                 
 dense_6 (Dense)             (None, 50)                2550      
                                                                 
 dense_7 (Dense)             (None, 6)                 306       
                                                                 
Total params: 10,956
Trainable params: 10,956
Non-trai

In [19]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = EpsGreedyQPolicy()
tf.compat.v1.disable_eager_execution()
dqn = DQNAgent(model=model,
              nb_actions=action_size,
              memory=memory,
              nb_steps_warmup=500,
              target_model_update=0.01, # 1e-2
              policy=policy)
dqn.compile(Adam, metrics=['mae'])
dqn.fit(env, nb_steps=10000,
       visualize=False,
       verbose=1,
       nb_max_episode_steps=20,
       log_interval=10000)

AttributeError: 'Sequential' object has no attribute '_compile_time_distribution_strategy'

Save Model

In [118]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [119]:
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


Load model from data

In [120]:
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [14]:
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [94]:
import time
import numpy as np
# from IPython.display import clear_output

In [95]:
def _get_action_for_state(state):
    # predicted = model.predict_on_batch(tf.expand_dims(state, axis=0))
    predicted = model.predict(tf.expand_dims(state, axis=0))
    print(predicted)
    if np.random.uniform(0, 1) < 0.6:
        action = np.random.choice([0,1,2,3,4,5])
    else:
        action = np.argmax(predicted)
    return action

In [96]:
sleep = 0.2
max_steps = 25


try:
    actions_str = ["South", "North", "East", "West", "Pickup", "Dropoff"]

    iteration = 0
    state = env.reset()  # reset environment to a new, random state
    env.render()
    print(f"Iter: {iteration} - Action: *** - Reward ***")
    time.sleep(sleep)
    done = False

    while not done:
        action = _get_action_for_state(state)
        iteration += 1
        state, reward, done, info = env.step(action)
        clear_output(wait=True)
        env.render()
        print(f"Iter: {iteration} - Action: {action}({actions_str[action]}) - Reward {reward}")
        time.sleep(sleep)
        if iteration == max_steps:
            print("cannot converge :(")
            break
except KeyboardInterrupt:
    pass

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
Iter: 25 - Action: 1(North) - Reward -1
cannot converge :(
